In [0]:
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60104281.dfs.core.windows.net",
    "9RU1c3jp0LFpRyz+fXUyQc4pc+RCoF+DolY45sCFCRkPKM4V4ucxzeXZMM247jLYqEkCXCfGxj6k+AStanhkHA=="
)

In [0]:
#Load the books dataset from the silver layer
books = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/processed/books/"
)
# Load the authors dataset from the silver layer
authors = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/processed/authors/"
)
# Display the first few records to confirm the data was loaded correctly
books.show(5)
authors.show(5)
# Display the columns and their data types to verify the schema
books.printSchema()
authors.printSchema()

+----------+------------------+------------+-------------+-------+----------+--------+--------------+-----------+--------------------+---------+--------------------+---------+----+--------------------+---------+---------------+-------------+-----------------+-------------------+----------------+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+
|      isbn|text_reviews_count|country_code|language_code|   name|      asin|is_ebook|average_rating|kindle_asin|         description|   format|                link|author_id|role|           publisher|num_pages|publication_day|       isbn13|publication_month|edition_information|publication_year|                 url|           image_url|book_id|ratings_count|work_id|               title|title_without_series|
+----------+------------------+------------+-------------+-------+----------+--------+--------------+-----------+--------------------+---------+--------------------+---------+---

In [0]:
from pyspark.sql.functions import col, length, trim, count, when
# Read raw (uncleaned) reviews from the silver layer
reviews = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/processed/reviews/")
# Peek at rows and schema
reviews.show(5, truncate=False)
reviews.printSchema()
# Basic profiling: counts and potential issues
total_rows = reviews.count()
null_review_id = reviews.filter(col("review_id").isNull()).count()
null_book_id = reviews.filter(col("book_id").isNull()).count()
null_user_id = reviews.filter(col("user_id").isNull()).count()
null_rating = reviews.filter(col("rating").isNull()).count()
empty_text = reviews.filter( (col("review_text").isNull()) | (trim(col("review_text")) ==
"") ).count()
print(f"Total rows: {total_rows}")
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id:{null_user_id}, NULL rating: {null_rating}")
print(f"Empty/NULL review_text: {empty_text}")

+--------------------------------+--------+--------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql.functions import col, trim, length

# Step 1 — Start from reviews DataFrame (already loaded from processed layer)
df = reviews

# Step 2 — Drop rows with missing critical identifiers
df = df.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)

# Step 3 — Clean and validate ratings
df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter(
    col("rating_int").isNotNull() &
    (col("rating_int") >= 1) &
    (col("rating_int") <= 5)
)

# Step 4 — Trim and clean review text
df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter(
    col("review_text").isNotNull() &
    (length(col("review_text")) >= 10)
)

# Step 5 — Remove duplicate reviews
df = df.dropDuplicates(["review_id"])

# Step 6 — Keep all relevant review-level columns
reviews_clean = df.select(
    "review_id",
    "book_id",
    "user_id",
    col("rating_int").alias("rating"),
    "review_text",
    "n_votes",
    "date_added"
)

In [0]:
# Write the cleaned reviews back to the silver layer (overwrite)
reviews_clean.write.mode("overwrite").parquet(
"abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/processed/reviews/"
)
# Sanity check: re-read from disk and inspect schema and a few rows
reviews_verified = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/processed/reviews/"
)
reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)
print(f"Written cleaned rows: {reviews_verified.count()}")

root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- date_added: string (nullable = true)

+--------------------------------+--------+--------------------------------+------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
spark.catalog.clearCache()


In [0]:
books = spark.read.parquet("abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/processed/books/")
authors = spark.read.parquet("abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/processed/authors/")
reviews_clean = spark.read.parquet("abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/processed/reviews/")

books.printSchema()
authors.printSchema()
reviews_clean.printSchema()


root
 |-- isbn: string (nullable = true)
 |-- text_reviews_count: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- is_ebook: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- kindle_asin: string (nullable = true)
 |-- description: string (nullable = true)
 |-- format: string (nullable = true)
 |-- link: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- role: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- num_pages: string (nullable = true)
 |-- publication_day: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- publication_month: string (nullable = true)
 |-- edition_information: string (nullable = true)
 |-- publication_year: string (nullable = true)
 |-- url: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- book_id: string (nullable = t

In [0]:
if "author_id" in books.columns:
    book_authors = books.select("book_id", "author_id").dropna(subset=["author_id"])
    print("Created bridge from books.author_id")

elif "authors" in books.columns:
    book_authors = books.withColumn("author_id", explode(col("authors"))).select("book_id", "author_id")
    print("Created bridge from exploded books.authors array")

elif "author_name" in books.columns:
    book_authors = books.join(
        authors,
        books["author_name"] == authors["name"],
        "inner"
    ).select(col("book_id"), col("author_id")).dropDuplicates()
    print("Created bridge by matching author_name → authors.name")

else:
    raise Exception("Could not infer author linkage. Please verify books schema.")

book_authors.printSchema()
book_authors.show(5, truncate=False)

# Save the bridge for future use
book_authors.write.mode("overwrite").parquet(
    "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/processed/book_authors/"
)
print("Saved book_authors bridge to processed zone")

# STEP 3: BUILD GOLD DATAFRAME
gold = (
    reviews_clean.alias("r")
    .join(books.alias("b"), col("r.book_id") == col("b.book_id"), "inner")
    .join(book_authors.alias("ba"), col("b.book_id") == col("ba.book_id"), "inner")
    .join(authors.alias("a"), col("ba.author_id") == col("a.author_id"), "left")
    .select(
        col("r.review_id"),
        col("r.book_id"),
        col("b.title"),
        col("a.author_id"),
        col("a.name"),
        col("r.user_id"),
        col("r.rating"),
        col("r.review_text"),
        col("b.language_code").alias("language"),
        col("r.n_votes"),
        col("r.date_added")
    )
)

print("Joined all tables successfully")

gold.printSchema()
gold.show(10, truncate=False)

Created bridge from books.author_id
root
 |-- book_id: string (nullable = true)
 |-- author_id: string (nullable = true)

+-------+---------+
|book_id|author_id|
+-------+---------+
|5333265|604031   |
|1333909|626222   |
|7327624|10333    |
|6066819|9212     |
|287140 |149918   |
+-------+---------+
only showing top 5 rows
Saved book_authors bridge to processed zone
Joined all tables successfully
root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- date_added: string (nullable = true)

+--------------------------------+--------+-----------------------------------------------------------------------------------+---------+----------------------

In [0]:
gold_path = "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/curated_reviews"

gold.write.format("delta").mode("overwrite").save(gold_path)
print("Gold (curated_reviews) Delta table written to:", gold_path)

Gold (curated_reviews) Delta table written to: abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/curated_reviews


In [0]:
# Drop if exists (for clean reruns)
spark.sql("DROP TABLE IF EXISTS curated_reviews")

# Register the existing Delta data folder as a managed table
spark.read.format("delta") \
    .load("abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/curated_reviews/") \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("curated_reviews")





In [0]:
spark.sql("SHOW TABLES").show(truncate=False)


+--------+---------------+-----------+
|database|tableName      |isTemporary|
+--------+---------------+-----------+
|default |curated_reviews|false      |
+--------+---------------+-----------+



In [0]:

spark.sql("SELECT * FROM curated_reviews LIMIT 10").show(truncate=False)
print(" Gold table ready and verified!")


+--------------------------------+--------+----------------------------------------------------------+---------+-------------------+--------------------------------+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------